# Category Based Recommender
In this notebook, I will be creating the recommender that only uses categorical data. I will do that using filters based on user inputs to give the best recommendations based on what the user is looking for.

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('../data/game_details.csv')
df.head()

,game_id,game_name,game_des,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,num_ratings,...,bayes_avg_rating,overall_rank_bayesavg,family_rank_bayes,complexity,categories,mechanics,families,implementations,designers,publishers
0,174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,2017,1,4,60,120,14,49737,...,8.49563,1|8.49563,thematic/1/8.47366|strategygames/1/8.46295,3.8728,Adventure|Exploration|Fantasy|Fighting|Miniatures,Action Queue|Action Retrieval|Campaign / Battl...,Category: Dungeon Crawler|Components: Miniatur...,NaN,Isaac Childres,Cephalofair Games|Albi|Albi Polska|Arclight|As...
1,161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,2015,2,4,60,60,13,46207,...,8.43544,2|8.43544,thematic/2/8.42624|strategygames/3/8.42193,2.8302,Environmental|Medical,Action Points|Cooperative Game|Hand Management...,Components: Map (Global Scale)|Components: Mul...,Pandemic Legacy: Season 2|Pandemic,Rob Daviau|Matt Leacock,Z-Man Games|Asterion Press|Devir|Filosofia Édi...
2,224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,2018,2,4,60,120,14,27629,...,8.42263,3|8.42263,strategygames/2/8.45557,3.9043,Economic|Industry / Manufacturing|Post-Napoleo...,Hand Management|Income|Loans|Market|Network an...,Cities: Birmingham (England)|Country: England|...,Brass: Lancashire,Gavan Brown|Matt Tolman|Martin Wallace,Roxley|Arclight|BoardM Factory|Conclave Editor...
3,167791,Terraforming Mars,"In the 2400s, mankind begins to terraform the ...",2016,1,5,120,120,12,77570,...,8.26819,5|8.26819,strategygames/6/8.24833,3.2478,Economic|Environmental|Industry / Manufacturin...,Closed Drafting|Contracts|Enclosure|End Game B...,Components: Map (Global Scale)|Components: Mul...,Terraforming Mars: Ares Expedition|Terraformin...,Jacob Fryxelius,FryxGames|Arclight|Brädspel.se|Fantasmagoria|G...
4,291457,Gloomhaven: Jaws of the Lion,Gloomhaven: Jaws of the Lion is a standalone g...,2020,1,4,30,120,14,18796,...,8.27022,4|8.27022,thematic/3/8.30864|strategygames/5/8.27021,3.5908,Adventure|Exploration|Fantasy|Fighting|Miniatures,Action Queue|Action Retrieval|Campaign / Battl...,Category: Dungeon Crawler|Components: Book as ...,NaN,Isaac Childres,Cephalofair Games|Albi|Albi Polska|Asmodee|Asm...


In [3]:
df.columns

Index(['game_id', 'game_name', 'game_des', 'game_yr_pub', 'min_players',
       'max_players', 'min_play_time', 'max_play_time', 'min_age',
       'num_ratings', 'avg_rating', 'bayes_avg_rating',
       'overall_rank_bayesavg', 'family_rank_bayes', 'complexity',
       'categories', 'mechanics', 'families', 'implementations', 'designers',
       'publishers'],
      dtype='object')

In [4]:
df.dtypes

game_id                    int64
game_name                 object
game_des                  object
game_yr_pub                int64
min_players                int64
max_players                int64
min_play_time              int64
max_play_time              int64
min_age                    int64
num_ratings                int64
avg_rating               float64
bayes_avg_rating         float64
overall_rank_bayesavg     object
family_rank_bayes         object
complexity               float64
categories                object
mechanics                 object
families                  object
implementations           object
designers                 object
publishers                object
dtype: object

### Checking and Addressing Any Nulls
This section focuses on addressing any and all nulls that may exist in the data frame.

In [5]:
df.isna().sum()

game_id                    0
game_name                  0
game_des                   0
game_yr_pub                0
min_players                0
max_players                0
min_play_time              0
max_play_time              0
min_age                    0
num_ratings                0
avg_rating                 0
bayes_avg_rating           0
overall_rank_bayesavg      0
family_rank_bayes          0
complexity                 0
categories                 3
mechanics                  0
families                   8
implementations          490
designers                  1
publishers                 0
dtype: int64

Outside of 'implementations', there are a few NaNs in 'categories', 'families', and 'designers'. I am going to go through each of those features and check on [boardgamegeek.com](https://boardgamegeek.com) to see if it was missed for some reason, or if it really is empty, figure out what is appropriate to fill in the NaNs with.

In [6]:
df[df['categories'].isna()]

,game_id,game_name,game_des,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,num_ratings,...,bayes_avg_rating,overall_rank_bayesavg,family_rank_bayes,complexity,categories,mechanics,families,implementations,designers,publishers
813,160851,Lanterns: The Harvest Festival,"The harvest is in, and the artisans are hard a...",2015,2,4,30,30,8,10378,...,6.66383,818|6.66383,familygames/217/6.7173,1.5617,NaN,Hand Management|Pattern Building|Set Collectio...,Country: China|Crowdfunding: Kickstarter|Game:...,Lanterns Dice: Lights in the Sky,Christopher Chung,Foxtrot Games|Renegade Game Studios|Broadway T...
824,106217,Hawaii,Game description from the publisher:&#10;&#10;...,2011,2,5,90,90,13,4350,...,6.65692,827|6.65692,strategygames/481/6.76548,2.9024,NaN,Action Points|Area Movement|Modular Board|Work...,Digital Implementations: Board Game Arena|Digi...,NaN,Greg Daigle,Hans im Glück|999 Games|Filosofia Éditions|Rio...
944,242684,Reavers of Midgard,Reavers of Midgard is a single worker placemen...,2019,2,4,60,120,14,2282,...,6.58044,944|6.58044,strategygames/506/6.73192,2.9091,NaN,Dice Rolling|Follow|Open Drafting|Set Collecti...,Admin: Better Description Needed!|Components: ...,NaN,J.B. Howell,Grey Fox Games|Arrakis Games|Corax Games|Don't...


For Lanterns: The Harvest Festival, the game is clearly a tile laying type game, but that is not a category. It also is not really a puzzle game. I don't want it to be without a category, so I am going with 'Abstract Strategy' since the rules appear to be straight forward and are their own thing. 

In [7]:
df.loc[813, 'categories'] = 'Abstract Strategy'

For Hawaii, it has elements of 'Territory Building' and 'Economic' categorical features. As such, I will be adding them to the game so it doesn't get filtered out.

In [8]:
df.loc[824, 'categories'] = 'Territory Building|Economic'

Reavers of Midgard squarely fits in the 'Medieval' and 'Nautical' categories. You could argue a couple of others, but because I personally have not played it and can only glean so much from online reviews, I am sticking with the two categories that seem like obvious fits to me.

In [9]:
df.loc[944, 'categories'] = 'Mdeieval|Nautical'

In [10]:
df.loc[[813,824,944], 'categories']

813              Abstract Strategy
824    Territory Building|Economic
944              Mdeieval|Nautical
Name: categories, dtype: object

In [11]:
df[df['families'].isna()]

,game_id,game_name,game_des,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,num_ratings,...,bayes_avg_rating,overall_rank_bayesavg,family_rank_bayes,complexity,categories,mechanics,families,implementations,designers,publishers
394,27162,Kingsburg,"In Kingsburg, players are Lords sent from the ...",2007,2,5,90,90,13,18888,...,7.01823,398|7.01823,strategygames/286/7.04281|familygames/87/7.05223,2.4254,City Building|Dice|Fantasy|Medieval,Dice Rolling|Worker Placement with Dice Workers,NaN,Kingsburg (Second Edition)|Kingsburg: The Dice...,Andrea Chiarvesio|Luca Iennaco,Stratelibri|Edge Entertainment|ElfinWerks|Fant...
451,202426,Sidereal Confluence,Sidereal Confluence: Trading and Negotiation i...,2017,4,9,120,180,14,3171,...,6.95954,447|6.95954,strategygames/240/7.13054,3.5504,Economic|Negotiation|Science Fiction,Auction/Bidding|Negotiation|Trading|Variable P...,NaN,NaN,TauCeti Deichmann,WizKids (I)|Banana Games|Frosted Games|Intrafi...
544,284435,Nova Luna,"The new moon is a symbol for a new beginning, ...",2019,1,4,30,60,8,4969,...,6.87599,546|6.87599,abstracts/16/7.1619,1.8667,Abstract Strategy|Puzzle,Contracts|Race|Tile Placement|Time Track,NaN,Framework|Habitats,Uwe Rosenberg|Corné van Moorsel,Edition Spielwiese|Brädspel.se|Djama Games|Gam...
595,296912,Fort,Fort is a 2-4 player card game about building ...,2020,2,4,20,40,10,5624,...,6.81637,598|6.81637,strategygames/384/6.8841,2.4286,Card Game,"Deck, Bag, and Pool Building|Follow|Hand Manag...",NaN,SPQF,Grant Rodiek,Leder Games|2Tomatoes Games|Asmodee|CrowD Game...
664,192457,Cry Havoc,"Cry Havoc is a card-driven, asymmetric, area c...",2016,2,4,60,120,10,6328,...,6.76725,664|6.76725,strategygames/418/6.84091,3.2707,Fighting|Miniatures|Science Fiction|Wargame,Area Majority / Influence|Area Movement|Auctio...,NaN,NaN,Grant Rodiek|Michał Oracz|Michał Walczak,Portal Games|Edge Entertainment|Galápagos Jogo...
823,301255,Whistle Mountain,Whistle Stop designer Scott Caputo has teamed ...,2020,2,4,60,90,14,1908,...,6.67203,804|6.67203,strategygames/399/6.86282,3.0000,Industry / Manufacturing,Tile Placement|Variable Player Powers|Worker P...,NaN,NaN,Scott Caputo|Luke Laurie,Bézier Games|BoardM Factory|La Boîte de Jeu|Ma...
881,199966,Kingsburg (Second Edition),The realm of Kingsburg is under attack! Monstr...,2017,2,5,90,120,10,2381,...,6.61488,886|6.61488,strategygames/493/6.7561,2.4493,City Building|Dice|Fantasy|Medieval,Dice Rolling|Worker Placement|Worker Placement...,NaN,Kingsburg|Kingsburg: To Forge a Realm,Andrea Chiarvesio|Luca Iennaco,Giochi Uniti|Stratelibri|Asmodee|Asmodee Korea...
882,207691,Railroad Revolution,Ambition and a thirst for wealth have driven h...,2016,2,4,45,90,12,2626,...,6.61148,893|6.61148,strategygames/484/6.76129,0.0000,Trains,Network and Route Building|Worker Placement|Wo...,NaN,NaN,Marco Canetta|Stefania Niccolini,What's Your Game?|hobbity.eu|Pegasus Spiele|Re...


Families is tricky because these are more detailed and specific to each game that I do not know having played none of them. Rather than guess at what might be best in a feature that I am leaning towards not implementing, I will fill the NaNs with blank strings. 

In [12]:
df.loc[df[df['families'].isna()].index, 'families'] = ''

In [13]:
df[df['designers'].isna()]

,game_id,game_name,game_des,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,num_ratings,...,bayes_avg_rating,overall_rank_bayesavg,family_rank_bayes,complexity,categories,mechanics,families,implementations,designers,publishers
879,165838,Space Hulk (Fourth Edition),From the depths of space an ancient vessel dri...,2014,2,2,60,60,12,1678,...,6.61582,884|6.61582,thematic/133/6.98528,2.7093,Fighting|Miniatures|Science Fiction,Action Points|Dice Rolling|Grid Movement|Modul...,Category: Dungeon Crawler|Game: Space Hulk|Pla...,Space Hulk (Third Edition),NaN,Games Workshop Ltd.


After looking online, this is the fourth edition of a game. There are no designers or artists listed online, and no designers for the third edition. As such, I am going to fill in `(Uncredited)` as the designer.

In [14]:
df.loc[879, 'designers'] = '(Uncredited)'
df.loc[879, 'designers']

'(Uncredited)'

The feature with the most NaNs, 'implementations', is going to be filled with blank string. 'Implementations' refers to when a game either uses the mechanics of a previous game or version of the same game, or is used as the base for a new game. If there is no implementation listed, then the game is a wholly unique version at this time.

In [15]:
df.fillna('', inplace=True)

In [16]:
df.isna().sum()

game_id                  0
game_name                0
game_des                 0
game_yr_pub              0
min_players              0
max_players              0
min_play_time            0
max_play_time            0
min_age                  0
num_ratings              0
avg_rating               0
bayes_avg_rating         0
overall_rank_bayesavg    0
family_rank_bayes        0
complexity               0
categories               0
mechanics                0
families                 0
implementations          0
designers                0
publishers               0
dtype: int64

### Generating Selection Lists
After reviewing all of the features that I have collected, I think the ones that people are most likely to use are `game_yr_pub`, `min_players`, `max_players`, `min_play_time`, `max_play_time`, `min_age`, `avg_rating`, `complexity`, `categories`, `mechanics`, `designers`, and `publishers`. I will also include the game's name and id features as well, but they can go to the game page on boardgamegeek.com if they want to learn more. I am going to be dropping the other columns and generating a list of all the options for each feature to create filters for the app. I am also adding in a `rank` column using the `top1000_updated.csv` since that has the rankings from when I started the project.

In [17]:
rankings = pd.read_csv('../data/top1000_updated.csv')
rankings.head()

,rank,title,id
0,1,Gloomhaven,174430
1,2,Pandemic Legacy: Season 1,161936
2,3,Brass: Birmingham,224517
3,4,Terraforming Mars,167791
4,5,Gloomhaven: Jaws of the Lion,291457


In [18]:
recom = df.drop(columns=['game_des', 'num_ratings', 'bayes_avg_rating',
                         'overall_rank_bayesavg', 'family_rank_bayes', 'families',
                         'implementations']).copy()
recom

,game_id,game_name,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,avg_rating,complexity,categories,mechanics,designers,publishers
0,174430,Gloomhaven,2017,1,4,60,120,14,8.72850,3.8728,Adventure|Exploration|Fantasy|Fighting|Miniatures,Action Queue|Action Retrieval|Campaign / Battl...,Isaac Childres,Cephalofair Games|Albi|Albi Polska|Arclight|As...
1,161936,Pandemic Legacy: Season 1,2015,2,4,60,60,13,8.58701,2.8302,Environmental|Medical,Action Points|Cooperative Game|Hand Management...,Rob Daviau|Matt Leacock,Z-Man Games|Asterion Press|Devir|Filosofia Édi...
2,224517,Brass: Birmingham,2018,2,4,60,120,14,8.65880,3.9043,Economic|Industry / Manufacturing|Post-Napoleo...,Hand Management|Income|Loans|Market|Network an...,Gavan Brown|Matt Tolman|Martin Wallace,Roxley|Arclight|BoardM Factory|Conclave Editor...
3,167791,Terraforming Mars,2016,1,5,120,120,12,8.41264,3.2478,Economic|Environmental|Industry / Manufacturin...,Closed Drafting|Contracts|Enclosure|End Game B...,Jacob Fryxelius,FryxGames|Arclight|Brädspel.se|Fantasmagoria|G...
4,291457,Gloomhaven: Jaws of the Lion,2020,1,4,30,120,14,8.64682,3.5908,Adventure|Exploration|Fantasy|Fighting|Miniatures,Action Queue|Action Retrieval|Campaign / Battl...,Isaac Childres,Cephalofair Games|Albi|Albi Polska|Asmodee|Asm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,94246,1812: The Invasion of Canada,2012,2,5,90,90,10,7.33553,2.1856,American Indian Wars|Educational|Wargame,Area Movement|Campaign / Battle Card Driven|Di...,Beau Beckett|Jeph Stahl,"Academy Games, Inc.|ASYNCRON games|Schwerkraft..."
996,163642,Chimera Station,2017,2,4,60,90,13,7.30728,3.1519,Science Fiction,Tile Placement|Turn Order: Claim Action|Variab...,Mark Major,Tasty Minstrel Games|Game Brewer
997,156091,Sons of Anarchy: Men of Mayhem,2014,3,4,90,90,13,7.27311,2.5577,Mafia|Movies / TV / Radio theme|Negotiation|Te...,Action Points|Area Majority / Influence|Dice R...,Aaron Dill|John Kovaleski|Sean Sweigart,"Gale Force Nine, LLC|Battlefront Miniatures Ltd"
998,24068,Shadow Hunters,2005,4,8,60,60,10,6.81905,1.8189,Adventure|Bluffing|Card Game|Deduction|Horror|...,Dice Rolling|Hidden Roles|Player Elimination|T...,Yasutaka Ikeda,"Game Republic, Inc.|cosaic|Giochi Uniti|Kaissa..."


In [19]:
recom = pd.merge(recom, rankings, left_on='game_id', right_on='id').drop(columns=['title', 'id'])
recom.head()

,game_id,game_name,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,avg_rating,complexity,categories,mechanics,designers,publishers,rank
0,174430,Gloomhaven,2017,1,4,60,120,14,8.72850,3.8728,Adventure|Exploration|Fantasy|Fighting|Miniatures,Action Queue|Action Retrieval|Campaign / Battl...,Isaac Childres,Cephalofair Games|Albi|Albi Polska|Arclight|As...,1
1,161936,Pandemic Legacy: Season 1,2015,2,4,60,60,13,8.58701,2.8302,Environmental|Medical,Action Points|Cooperative Game|Hand Management...,Rob Daviau|Matt Leacock,Z-Man Games|Asterion Press|Devir|Filosofia Édi...,2
2,224517,Brass: Birmingham,2018,2,4,60,120,14,8.65880,3.9043,Economic|Industry / Manufacturing|Post-Napoleo...,Hand Management|Income|Loans|Market|Network an...,Gavan Brown|Matt Tolman|Martin Wallace,Roxley|Arclight|BoardM Factory|Conclave Editor...,3
3,167791,Terraforming Mars,2016,1,5,120,120,12,8.41264,3.2478,Economic|Environmental|Industry / Manufacturin...,Closed Drafting|Contracts|Enclosure|End Game B...,Jacob Fryxelius,FryxGames|Arclight|Brädspel.se|Fantasmagoria|G...,4
4,291457,Gloomhaven: Jaws of the Lion,2020,1,4,30,120,14,8.64682,3.5908,Adventure|Exploration|Fantasy|Fighting|Miniatures,Action Queue|Action Retrieval|Campaign / Battl...,Isaac Childres,Cephalofair Games|Albi|Albi Polska|Asmodee|Asm...,5


In [20]:
recom['game_yr_pub'].min(), recom['game_yr_pub'].max()
# There appears to be a -2200 for a game year, so I am looing into that a bit.

(-2200, 2021)

In [21]:
recom.sort_values('game_yr_pub').head(10)
# These make sense now. Going to leave it alone for now. I may have to get creative in the app later

,game_id,game_name,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,avg_rating,complexity,categories,mechanics,designers,publishers,rank
189,188,Go,-2200,2,2,30,180,8,7.63939,3.9386,Abstract Strategy,Enclosure|Square Grid,(Uncredited),(Public Domain)|(Self-Published)|(Unknown)|3M|...,190
432,171,Chess,1475,2,2,0,0,6,7.16095,3.6826,Abstract Strategy,Grid Movement|Pattern Movement|Square Grid|Sta...,(Uncredited),(Public Domain)|(Self-Published)|(Unknown)|3M|...,433
640,2398,Cribbage,1630,2,4,30,30,10,7.07419,1.9024,Card Game,Hand Management|Push Your Luck,Sir John Suckling,(Public Domain)|(Unknown)|5th Peg|Acme Ruler C...,641
732,45,Perudo,1800,2,6,15,30,8,6.90609,1.2605,Bluffing|Dice|Movies / TV / Radio theme|Party ...,Betting and Bluffing|Dice Rolling|Player Elimi...,Richard Borg,(Public Domain)|Endless Games (I)|F.X. Schmid|...,733
821,2093,Mahjong,1850,3,4,120,120,8,7.03181,2.5681,Abstract Strategy,Hand Management|Set Collection,(Uncredited),(Public Domain)|(Unknown)|A. D. Richter & Cie....,822
58,521,Crokinole,1876,2,4,30,30,8,7.96301,1.2480,Action / Dexterity,Flicking|Team-Based Game,(Uncredited),(Public Domain)|(Unknown)|BrownCastle Games|Ca...,59
793,2181,Bridge,1925,4,4,60,60,12,7.45893,3.8610,Card Game,Auction/Bidding|Hand Management|Predictive Bid...,Harold Vanderbilt,(Public Domain)|Ariel Productions Ltd|ASS Alte...,794
665,483,Diplomacy,1959,2,7,360,360,12,7.04085,3.3493,Bluffing|Negotiation|Political|Post-Napoleonic...,Area Majority / Influence|Area Movement|Negoti...,Allan B. Calhamer,"The Avalon Hill Game Co|Games Research, Inc.|(...",666
299,5,Acquire,1964,2,6,90,90,12,7.33844,2.5003,Economic|Territory Building,Hand Management|Investment|Market|Square Grid|...,Sid Sackson,"3M|The Avalon Hill Game Co|Avalon Hill Games, ...",300
706,1035,Squad Leader,1977,2,2,60,60,14,7.53212,4.0363,Wargame|World War II,Critical Hits and Failures|Dice Rolling|Events...,John Hill (I),"Arsenal Publishing, Inc.|The Avalon Hill Game ...",707


In [22]:
recom['min_players'].value_counts(), 

(2    614
 1    286
 3     79
 4     16
 5      4
 6      1
 Name: min_players, dtype: int64,)

In [23]:
recom['max_players'].value_counts()
# I think having the max option being 11+ will cover this.

4      449
5      234
2      114
6      110
8       32
7       21
10      10
3        9
1        7
12       6
100      2
18       2
16       1
9        1
99       1
30       1
Name: max_players, dtype: int64

In [24]:
recom['min_play_time'].value_counts(bins=[0, 15, 30, 45, 60, 480])
# These bins help narrow things down and aren't overwhelming.

(15.0, 30.0]      259
(60.0, 480.0]     256
(45.0, 60.0]      254
(30.0, 45.0]      175
(-0.001, 15.0]     56
Name: min_play_time, dtype: int64

In [25]:
recom['max_play_time'].value_counts(bins=[0, 30, 60, 90, 120, 1250])
# These bins will work if max time is used.

(30.0, 60.0]       325
(60.0, 90.0]       176
(-0.001, 30.0]     170
(90.0, 120.0]      169
(120.0, 1250.0]    160
Name: max_play_time, dtype: int64

In [26]:
recom['min_age'].value_counts().sort_index()

0       2
4       1
5       2
6       5
7       9
8     144
9      20
10    173
11      4
12    263
13    119
14    247
15      4
16      1
17      5
18      1
Name: min_age, dtype: int64

In [27]:
recom['avg_rating'].min(), recom['avg_rating'].max()

(6.6688, 8.81379)

In [28]:
recom['complexity'].min(), recom['complexity'].max()

(0.0, 4.7359)

In [29]:
recom[recom['complexity'] == 0.0]
# These games do have weights (what I call `complexity`), so I will fill them in.

,game_id,game_name,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,avg_rating,complexity,categories,mechanics,designers,publishers,rank
115,2511,Sherlock Holmes Consulting Detective: The Tham...,1982,1,8,60,120,13,7.69245,0.0,Deduction|Murder/Mystery|Novel-based|Post-Napo...,Cooperative Game|Narrative Choice / Paragraph|...,Raymond Edwards|Suzanne Goldberg|Gary Grady,Sleuth Publications|Space Cowboys|Angry Lion G...,116
118,175155,Forbidden Stars,2015,2,4,120,180,14,7.96961,0.0,Bluffing|Fighting|Miniatures|Science Fiction|W...,Area Majority / Influence|Area Movement|Card P...,Samuel Bailey|James Kniffen|Corey Konieczka,Fantasy Flight Games|Asterion Press|Edge Enter...,119
177,189932,Tyrants of the Underdark,2016,2,4,60,60,14,7.91580,0.0,Fantasy|Fighting,"Area Majority / Influence|Connections|Deck, Ba...",Peter Lee|Rodney Thompson|Andrew Veen,"Gale Force Nine, LLC|Heidelberger Spieleverlag...",178
191,12,Ra,1999,2,5,45,60,12,7.48443,0.0,Ancient|Mythology,Auction/Bidding|Auction: Once Around|Closed Ec...,Reiner Knizia,alea|Ravensburger|25th Century Games|999 Games...,192
198,209418,Dominion (Second Edition),2016,2,4,30,30,14,7.89540,0.0,Card Game|Medieval,"Deck, Bag, and Pool Building|Hand Management|O...",Donald X. Vaccarino,Rio Grande Games|999 Games|Conclave Editora|De...,199
270,48726,Alien Frontiers,2010,2,4,90,90,13,7.39179,0.0,Dice|Science Fiction,Area Majority / Influence|Dice Rolling|Open Dr...,Tory Niemann,Clever Mojo Games|Game Salute|Hobby World|LocW...,271
279,139976,Cthulhu Wars,2015,2,4,90,120,14,7.89068,0.0,Fantasy|Fighting|Horror|Miniatures,Action Points|Area Majority / Influence|Area M...,Sandy Petersen|Lincoln Petersen,Petersen Games|Arclight|BoardM Factory|Fabrika...,280
291,172818,Above and Below,2015,2,4,90,90,13,7.39490,0.0,Adventure|City Building|Economic|Exploration|F...,Action Points|Dice Rolling|Narrative Choice / ...,Ryan Laukat,Red Raven Games|Arclight|Bard Centrum Gier|Con...,292
304,176920,Mission: Red Planet (Second Edition),2015,2,6,45,90,14,7.43791,0.0,Science Fiction,Action Retrieval|Area Majority / Influence|Are...,Bruno Cathala|Bruno Faidutti,Fantasy Flight Games|Edge Entertainment|Galakt...,305
312,163745,Star Wars: Armada,2015,2,2,120,120,14,7.83425,0.0,Collectible Components|Miniatures|Movies / TV ...,Action Queue|Critical Hits and Failures|Dice R...,James Kniffen|Christian T. Petersen,Fantasy Flight Games|Edge Entertainment|Galakt...,313


In [30]:
fix = recom[recom['complexity'] == 0.0].index
fix

Int64Index([115, 118, 177, 191, 198, 270, 279, 291, 304, 312, 332, 338, 365,
            382, 411, 418, 427, 460, 509, 530, 574, 594, 596, 718, 728, 784,
            831, 849, 882],
           dtype='int64')

In [31]:
recom.loc[115, 'complexity'] = 2.67
recom.loc[118, 'complexity'] = 3.83
recom.loc[177, 'complexity'] = 2.55
recom.loc[191, 'complexity'] = 2.35
recom.loc[198, 'complexity'] = 2.16
recom.loc[270, 'complexity'] = 2.57
recom.loc[279, 'complexity'] = 3.15
recom.loc[291, 'complexity'] = 2.52
recom.loc[304, 'complexity'] = 2.20
recom.loc[312, 'complexity'] = 3.15
recom.loc[332, 'complexity'] = 3.10
recom.loc[338, 'complexity'] = 2.16
recom.loc[365, 'complexity'] = 2.78
recom.loc[382, 'complexity'] = 2.96
recom.loc[411, 'complexity'] = 2.74
recom.loc[418, 'complexity'] = 3.34
recom.loc[427, 'complexity'] = 3.63
recom.loc[460, 'complexity'] = 3.12
recom.loc[509, 'complexity'] = 3.38
recom.loc[530, 'complexity'] = 3.80
recom.loc[574, 'complexity'] = 3.55
recom.loc[594, 'complexity'] = 2.58
recom.loc[596, 'complexity'] = 1.48
recom.loc[718, 'complexity'] = 2.07
recom.loc[728, 'complexity'] = 3.09
recom.loc[784, 'complexity'] = 1.02
recom.loc[831, 'complexity'] = 2.19
recom.loc[849, 'complexity'] = 2.08
recom.loc[882, 'complexity'] = 3.26

In [32]:
recom.loc[fix]

,game_id,game_name,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,avg_rating,complexity,categories,mechanics,designers,publishers,rank
115,2511,Sherlock Holmes Consulting Detective: The Tham...,1982,1,8,60,120,13,7.69245,2.67,Deduction|Murder/Mystery|Novel-based|Post-Napo...,Cooperative Game|Narrative Choice / Paragraph|...,Raymond Edwards|Suzanne Goldberg|Gary Grady,Sleuth Publications|Space Cowboys|Angry Lion G...,116
118,175155,Forbidden Stars,2015,2,4,120,180,14,7.96961,3.83,Bluffing|Fighting|Miniatures|Science Fiction|W...,Area Majority / Influence|Area Movement|Card P...,Samuel Bailey|James Kniffen|Corey Konieczka,Fantasy Flight Games|Asterion Press|Edge Enter...,119
177,189932,Tyrants of the Underdark,2016,2,4,60,60,14,7.91580,2.55,Fantasy|Fighting,"Area Majority / Influence|Connections|Deck, Ba...",Peter Lee|Rodney Thompson|Andrew Veen,"Gale Force Nine, LLC|Heidelberger Spieleverlag...",178
191,12,Ra,1999,2,5,45,60,12,7.48443,2.35,Ancient|Mythology,Auction/Bidding|Auction: Once Around|Closed Ec...,Reiner Knizia,alea|Ravensburger|25th Century Games|999 Games...,192
198,209418,Dominion (Second Edition),2016,2,4,30,30,14,7.89540,2.16,Card Game|Medieval,"Deck, Bag, and Pool Building|Hand Management|O...",Donald X. Vaccarino,Rio Grande Games|999 Games|Conclave Editora|De...,199
270,48726,Alien Frontiers,2010,2,4,90,90,13,7.39179,2.57,Dice|Science Fiction,Area Majority / Influence|Dice Rolling|Open Dr...,Tory Niemann,Clever Mojo Games|Game Salute|Hobby World|LocW...,271
279,139976,Cthulhu Wars,2015,2,4,90,120,14,7.89068,3.15,Fantasy|Fighting|Horror|Miniatures,Action Points|Area Majority / Influence|Area M...,Sandy Petersen|Lincoln Petersen,Petersen Games|Arclight|BoardM Factory|Fabrika...,280
291,172818,Above and Below,2015,2,4,90,90,13,7.39490,2.52,Adventure|City Building|Economic|Exploration|F...,Action Points|Dice Rolling|Narrative Choice / ...,Ryan Laukat,Red Raven Games|Arclight|Bard Centrum Gier|Con...,292
304,176920,Mission: Red Planet (Second Edition),2015,2,6,45,90,14,7.43791,2.20,Science Fiction,Action Retrieval|Area Majority / Influence|Are...,Bruno Cathala|Bruno Faidutti,Fantasy Flight Games|Edge Entertainment|Galakt...,305
312,163745,Star Wars: Armada,2015,2,2,120,120,14,7.83425,3.15,Collectible Components|Miniatures|Movies / TV ...,Action Queue|Critical Hits and Failures|Dice R...,James Kniffen|Christian T. Petersen,Fantasy Flight Games|Edge Entertainment|Galakt...,313


In [33]:
recom['complexity'].min(), recom['complexity'].max()

(1.02, 4.7359)

In [34]:
categories = []
for row in recom['categories']:
    categories += row.split('|')
set(categories)

{'Abstract Strategy',
 'Action / Dexterity',
 'Adventure',
 'Age of Reason',
 'American Civil War',
 'American Indian Wars',
 'American Revolutionary War',
 'American West',
 'Ancient',
 'Animals',
 'Arabian',
 'Aviation / Flight',
 'Bluffing',
 'Card Game',
 "Children's Game",
 'City Building',
 'Civil War',
 'Civilization',
 'Collectible Components',
 'Comic Book / Strip',
 'Deduction',
 'Dice',
 'Economic',
 'Educational',
 'Electronic',
 'Environmental',
 'Expansion for Base-game',
 'Exploration',
 'Fantasy',
 'Farming',
 'Fighting',
 'Horror',
 'Humor',
 'Industry / Manufacturing',
 'Mafia',
 'Math',
 'Mature / Adult',
 'Maze',
 'Mdeieval',
 'Medical',
 'Medieval',
 'Memory',
 'Miniatures',
 'Modern Warfare',
 'Movies / TV / Radio theme',
 'Murder/Mystery',
 'Mythology',
 'Napoleonic',
 'Nautical',
 'Negotiation',
 'Novel-based',
 'Number',
 'Party Game',
 'Pike and Shot',
 'Pirates',
 'Political',
 'Post-Napoleonic',
 'Prehistoric',
 'Print & Play',
 'Puzzle',
 'Racing',
 'Real-t

In [35]:
mechanics = []
for row in recom['mechanics']:
    mechanics += row.split('|')
set(mechanics)

{'Acting',
 'Action Drafting',
 'Action Points',
 'Action Queue',
 'Action Retrieval',
 'Action Timer',
 'Action/Event',
 'Advantage Token',
 'Alliances',
 'Area Majority / Influence',
 'Area Movement',
 'Area-Impulse',
 'Auction/Bidding',
 'Auction: Dutch',
 'Auction: Dutch Priority',
 'Auction: English',
 'Auction: Fixed Placement',
 'Auction: Once Around',
 'Auction: Sealed Bid',
 'Auction: Turn Order Until Pass',
 'Automatic Resource Growth',
 'Betting and Bluffing',
 'Bias',
 'Bingo',
 'Bribery',
 'Campaign / Battle Card Driven',
 'Card Play Conflict Resolution',
 'Catch the Leader',
 'Chaining',
 'Closed Drafting',
 'Closed Economy Auction',
 'Command Cards',
 'Commodity Speculation',
 'Communication Limits',
 'Connections',
 'Constrained Bidding',
 'Contracts',
 'Cooperative Game',
 'Critical Hits and Failures',
 'Cube Tower',
 'Deck Construction',
 'Deck, Bag, and Pool Building',
 'Deduction',
 'Delayed Purchase',
 'Dice Rolling',
 'Die Icon Resolution',
 'Different Dice Moveme

In [36]:
designers = []
for row in recom['designers']:
    designers += row.split('|')
set(designers)

{'(Uncredited)',
 'Aaron Dill',
 'Aaron Weissblum',
 'Acchittocca',
 'Adam Carlson',
 'Adam Kałuża',
 'Adam Kwapiński',
 'Adam P. McIver',
 'Adam Poots',
 'Adam Rebottaro',
 'Adam Sadler',
 'Adam Spanel',
 'Adrian Abela',
 'Adrian Adamescu',
 'Akihisa Okui',
 'Alain Orban',
 'Alain Rivollet',
 'Alan Gerding',
 'Alan R. Moon',
 'Alan Roach',
 'Alan Stone',
 'Alban Viard',
 'Alex Davy',
 'Alex Hague',
 'Alex Randolph',
 'Alex Rockwell',
 'Alexandar Ortloff',
 'Alexander Pfister',
 'Alexander Schmidt (II)',
 'Alexandr Ushan',
 'Allan B. Calhamer',
 'Alvydas Jakeliunas',
 'Amabel Holland',
 'Ananda Gupta',
 'Andrea Chiarvesio',
 'Andrea Crespi',
 'Andreas "ode." Odendahl',
 'Andreas Pelikan',
 'Andreas Schmidt',
 'Andreas Seyfarth',
 'Andreas Steding',
 'Andreas Steiger',
 'Andrew Looney',
 'Andrew Nerger',
 'Andrew Parks',
 'Andrew Ruhnke',
 'Andrew Veen',
 'Andrew Wolf',
 'André Zatz',
 'Andy Clautice',
 'Andy Hoare',
 'Andy Jones',
 'Annick Lobet',
 'Anthony J. Gallela',
 'Antoine Bauza

In [37]:
publishers = []
for row in recom['publishers']:
    publishers += row.split('|')
set(publishers)

{'Phantasia Co., Ltd',
 'Piet Hein A/S',
 'Nova Era (Нова Ера)',
 'Quined Games',
 'FoxGames',
 'Allied Van Lines',
 'Toy Brokers Ltd',
 'Two Lanterns Games',
 'Gametrade Distribuzione',
 "Surfin' Meeple",
 'Fantasy Flight Games',
 'Devir',
 'The Wood Games',
 'Millenium',
 'Common Man Games',
 '(Web published)',
 'Fractal Juegos',
 'Restoration Games',
 'Tulip Games',
 'XGen Studios',
 'Бельвіль',
 'James Ernest Games',
 'Bézier Games',
 'eggertspiele',
 'APE Games',
 'Ulisses Spiele',
 'Logojogos',
 'Imaginarium',
 'Game Factory',
 'MB Spellen',
 'Kod Kod',
 'Quined White Goblin Games',
 'Jumbo',
 'Brotherwise Games',
 'Ludically',
 'Blatz',
 'Arrco Playing Card Co.',
 'KOZAK Games',
 'After 5 Games',
 'Crown & Andrews Ltd.',
 'NeoTroy Games',
 'Asmodee Spain',
 'Lui-même',
 'Takara',
 'Vennerød Forlag AS',
 'Microsoft Game Studios',
 "OPEN'N PLAY",
 'Michael Stanfield',
 'Estrela',
 'Chrononauts Games',
 'YingDi (旅法师营地)',
 'Sunnygeeks',
 'G3',
 'ElfinWerks',
 'Second Gate Games',
 '

### Creating Function to Filter Data Frames
In this section I will create the function to filter the data frame to give the recommendations

In [94]:
def category_filter(yr=None, play=None, ptime = None,
                    age=None, comp=None, cat=[],
                    des=None, pubs=None, rate=None):
    res = recom
    # Filter by year
    if yr != None:
        res = res[res['game_yr_pub'] == yr]
    # Filter by number of players
    if play != None:
        res = res[(res['min_players'] <= play) & (res['max_players'] >= play)]
    # Filter by play time
    if ptime != None:
        if ptime == '0 - 15':
            res = res[res['min_play_time'] <= 15]
        elif ptime == '15 - 30':
            res = res[(res['min_play_time'] > 15) & (res['min_play_time'] <= 30)]
        elif ptime == '30 - 45':
            res = res[(res['min_play_time'] > 30) & (res['min_play_time'] <= 45)]
        elif ptime == '45 - 60':
            res = res[(res['min_play_time'] > 45) & (res['min_play_time'] <= 60)]
        else:
            res = res[res['min_play_time'] > 60]
    # Filter by age constraint
    if age != None:
        res = res[res['min_age'] <= age]
    # Filter by average rating
    if rate != None:
        res = res[res['avg_rating'] >= rate]
    # Filter by complexity
    if comp != None:
        res = res[(res['complexity'] >= (comp - .5)) & (res['complexity'] <= (comp + .5))]
    # Filter by categories
#     if len(cat) != 0:
#         res.reset_index(inplace=True)
        
    # Filter by designers
    # Filter by publishers
    return res

In [98]:
category_filter(yr=2017,
                play=4,
                ptime='15 - 30',
                age=14,
                rate=7.5,
                comp=2,
                cat=[],
                des=None, 
                pubs=None)

,game_id,game_name,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,avg_rating,complexity,categories,mechanics,designers,publishers,rank
60,230802,Azul,2017,2,4,30,45,8,7.79172,1.7658,Abstract Strategy|Puzzle|Renaissance,End Game Bonuses|Open Drafting|Pattern Buildin...,Michael Kiesling,Next Move Games|Plan B Games|Asmodee|Broadway ...,61
148,217372,The Quest for El Dorado,2017,2,4,30,60,10,7.63370,1.9284,Adventure|Exploration|Racing,"Deck, Bag, and Pool Building|Grid Movement|Han...",Reiner Knizia,Ravensburger|999 Games|Arclight|Brädspel.se|Ga...,149
168,199561,Sagrada,2017,1,4,30,45,14,7.50891,1.9265,Dice|Puzzle,Contracts|Dice Rolling|End Game Bonuses|Grid C...,Adrian Adamescu|Daryl Andrews,Floodgate Games|Cranio Creations|CrowD Games|D...,169
210,232832,Century: Golem Edition,2017,2,5,30,45,8,7.67247,1.6825,Card Game|Economic|Fantasy,"Action Retrieval|Contracts|Deck, Bag, and Pool...",Emerson Matsuuchi,Plan B Games|Arclight|Boardgame Space|Cube Fac...,211
258,223040,Fantasy Realms,2017,2,6,20,20,14,7.56338,1.7421,Card Game|Fantasy,Open Drafting|Set Collection,Bruce Glassco,WizKids (I)|BoardM Factory|Broadway Toys LTD|D...,259


In [ ]:
recom.reset_index()

In [64]:
cat_list = list(sorted(set(categories)))

['Abstract Strategy',
 'Action / Dexterity',
 'Adventure',
 'Age of Reason',
 'American Civil War',
 'American Indian Wars',
 'American Revolutionary War',
 'American West',
 'Ancient',
 'Animals',
 'Arabian',
 'Aviation / Flight',
 'Bluffing',
 'Card Game',
 "Children's Game",
 'City Building',
 'Civil War',
 'Civilization',
 'Collectible Components',
 'Comic Book / Strip',
 'Deduction',
 'Dice',
 'Economic',
 'Educational',
 'Electronic',
 'Environmental',
 'Expansion for Base-game',
 'Exploration',
 'Fantasy',
 'Farming',
 'Fighting',
 'Horror',
 'Humor',
 'Industry / Manufacturing',
 'Mafia',
 'Math',
 'Mature / Adult',
 'Maze',
 'Mdeieval',
 'Medical',
 'Medieval',
 'Memory',
 'Miniatures',
 'Modern Warfare',
 'Movies / TV / Radio theme',
 'Murder/Mystery',
 'Mythology',
 'Napoleonic',
 'Nautical',
 'Negotiation',
 'Novel-based',
 'Number',
 'Party Game',
 'Pike and Shot',
 'Pirates',
 'Political',
 'Post-Napoleonic',
 'Prehistoric',
 'Print & Play',
 'Puzzle',
 'Racing',
 'Real-t

In [68]:
test_str = 'Abstract Strategy|Puzzle|Renaissance'
test_list = ['Abstract Strategy', 'Renaissance']

In [103]:
for row in recom[]:
    print(row)

categories
game_name


In [104]:
recom[['categories', 'game_name']]

,categories,game_name
0,Adventure|Exploration|Fantasy|Fighting|Miniatures,Gloomhaven
1,Environmental|Medical,Pandemic Legacy: Season 1
2,Economic|Industry / Manufacturing|Post-Napoleo...,Brass: Birmingham
3,Economic|Environmental|Industry / Manufacturin...,Terraforming Mars
4,Adventure|Exploration|Fantasy|Fighting|Miniatures,Gloomhaven: Jaws of the Lion
...,...,...
995,American Indian Wars|Educational|Wargame,1812: The Invasion of Canada
996,Science Fiction,Chimera Station
997,Mafia|Movies / TV / Radio theme|Negotiation|Te...,Sons of Anarchy: Men of Mayhem
998,Adventure|Bluffing|Card Game|Deduction|Horror|...,Shadow Hunters


In [85]:
recom[recom['categories'].str.split('|').isin(['Puzzle'])]

,game_id,game_name,game_yr_pub,min_players,max_players,min_play_time,max_play_time,min_age,avg_rating,complexity,categories,mechanics,designers,publishers,rank


In [107]:
recom.drop(columns=['categories', 'mechanics', 'designers', 'publishers', 'max_play_time']).to_csv('../streamlit_2/test.csv', index=False)